## Task 3 - Hypothesis Testing

Repeat Task 3.2 after rescaling all features in the range [10, 15]. Use hypothesis testing to compare the performance of the models developed in this task with that developed in Tasks 3.2.

Markdown Question: What was the hypothesis you rejected in this task? How does p-value affect your decision?

<span style="font-weight:bold;">Save your response (program and markdown) as task3.ipynb</span>

hypotheses testing for classification

In [ ]:
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt 
from functions.pre_processing import *
from functions.transformations import *
from functions.feature_selection import *
from functions.analysis import *
from functions.classifiers import *
from functions.evaluation import *
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
from sklearn.exceptions import ConvergenceWarning

-----
## Pre-Processing and Transformation

In [2]:
x_train, x_test, y_train, y_test = fetch_datasets()

x_train_norm, x_test_norm = normalise_min_max_task_3(x_train, x_test)

---
## MLP Classifier Training

In [4]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)
mlp_training_holdout = mlp_classifier(x_train_norm, x_test_norm, y_train, y_test, "holdout")
mlp_training_cv = mlp_classifier(x_train_norm, x_test_norm, y_train, y_test, "cv")
mlp_training_k_fold = mlp_classifier(x_train_norm, x_test_norm, y_train, y_test, "k_fold")
mlp_training_k_fold_strat = mlp_classifier(x_train_norm, x_test_norm, y_train, y_test, "k_fold_strat")

------ Holdout Validation ------
Training Accuracy: 0.115
Testing Accuracy: 0.10796296296296297
------ Cross Validation ------
Mean Accuracy: 0.12117283950617283
------ K fold Validation ------
Mean Accuracy: 0.12086419753086419
Std Deviation: 0.036688483346172016
------ Stratified K fold Validation ------
Mean Accuracy: 0.13228395061728396
Std Deviation: 0.06045353281347278


---
## Hypothesis Testing 

In [ ]:
# Testing this model and the one from the previous [TO DO]
# Classification accuracy
# TP, TN, FP, FN
# Precision, Recall, F-score, support
# Assuming TP rate is 75%, How confident are you that the same TP rate will be achieved when the model is used in a real application?
# How do you statistically validate conclusions like “SVM is better than linear regression”?
# T-test -> 1 sample t-test, 2 sample t-test (relevant as it is used to compaee 2 models), paired t-test


In [6]:
# Task 2 rough
x_train, x_test, y_train, y_test = fetch_datasets()

x_train_norm, x_test_norm = normalise_min_max(x_train, x_test)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# y_pred = mlp.predict(x_test_norm)
# confusion_matrix(y_test, y_pred)
# FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
# FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
# TP = np.diag(confusion_matrix)
# TN = confusion_matrix.values.sum() - (FP + FN + TP)


Markdown Question: What was the hypothesis you rejected in this task? How does p-value affect your decision?